# Huawei Research France

In [138]:
import pandas as pd
import numpy as np
import rampwf as rw
import datetime
import time

In [124]:
problem = rw.utils.assert_read_problem()

Ajout des données

In [125]:
X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



Fonctions temporelles

In [126]:
def get_unit(row, day=True, hour=True, minute=True):
    """
    Crée une nouvelle variable à partir de l'index de temps d'un df.
    Les valeurs sont des unités de temps et non des 
    jours/heures/minutes réelles.
    
    Paramètres
        - day : par defaut True, ajoute le jour 
        - hour : par defaut True, ajoute l'heure 
        - minute : par defaut True, ajoute la minute 
        
    """
    if day:
        row["day"] = 'D' + str(row.name.day)
    if hour:
        row["hour"] = 'H' + str(row.name.hour) 
    if minute:
        row['minute'] = 'M' + str(row.name.minute)
    return row

In [127]:
def _get_add_unit(add_unit):
    params = {"day": False,
              'hour': False,
              'minute': False}
    
    for i in add_unit:
        if i in params:
            params[i] = True
    return params

In [128]:
def _resampling(X, resample, columns, verbose=True):
    if verbose:
        print(f'Resampling with {resample}')
    
    if isinstance(resample, list):
        liste_r = []
        for r in resample:
            if 'unit' in r and 'func' in r:
                s_X = getattr(X.resample(r["unit"]), r["func"])()
                s_X.columns = [c + '-' + r['func'] for c in columns]
                liste_r.append(s_X)
                
            else:
                raise KeyError("Invalid key must have {'unit', 'func'}")
        X = pd.concat(liste_r, axis=1)
                
    else:
        if 'unit' in resample and 'func' in resample:
            X = getattr(X.resample(resample["unit"]), resample["func"])()
        else:
            raise KeyError("Invalid key must have {'unit', 'func'}")
    return X

In [129]:
def create_df_obs(X, y=None,
                  source='source',
                  sample=44,
                  extra='',
                  add_unit=["day", "hour", "minute"],
                  resample=None, 
                  verbose=True):
    """
    Permet de récupérer une observation et son label
    sur une source spéficique de données. Possibilité 
    de :
        - choisir un resampling temporelle
        - ajouter des variables temporelles

    Paramètres
        - X : OpticalDataset
        - y : OpticalLabels
        - source : défaut 'source', possible valeurs : 
                    [source', 'source_bkg','target',
                    'target_bkg', 'target_unlabeled']
        - extra : par défaut '', ajoute un id aux colonnes
        - add_unit : Ajoute une colonne temporelle parmi 
                    ["day", "hour", "minute"], par défaut toutes
        - sample : int, par défaut 44, numéro de l'observation
        - resample : défaut None, requiert un dictionnaire : 
                    {"unit": '<unité de temps>', 
                     "func": '<fonction d'aggrégration>'}
        - verbose : affiche les actions effectuées

    Return
        X, y : pd.Dataframe de l'observation et son label
    """
    if y is not None:
        if source in ['source', 'target']:
            y = getattr(y, source)[sample]
        else:
            y = None

    # permet de créer une datetime 1 janvier 2000 à 00:00 et
    # selectionne l'observation (feature x unité de temps).
    # Fixe ensuite un index de 15 min entre chaque unité

    start = datetime.datetime(2000, 1, 1, 0)
    obs = getattr(X, source)[sample]
    index = pd.date_range(start, periods=len(obs), freq="15T")

    columns = ["current", 
               "err_down_bip",
               "err_up_bip",
               "olt_recv",
               "rdown",
               "recv",
               "rup",
               "send",
               "temp", 
               "volt"]
    
    columns = [c + extra for c in columns]

    X = pd.DataFrame(obs, index=index,
                     columns=columns)

    if resample:
        X = _resampling(X=X, resample=resample, columns=columns, verbose=verbose)

    add_unit = _get_add_unit(add_unit)
    X = X.apply(get_unit, **add_unit, axis=1)
    return X, y

In [146]:
def flatten_df(X, name=None):
    add_unit = ['day', 'hour', 'minute']
    col_names = []        
    for i in X.index:
        for c in X.columns:
            if c in add_unit:
                col_names.append(c)
            else:
                col_names.append(f'{i} {c}')
    tmp = pd.DataFrame(X.stack(dropna=False).values).T
    tmp.columns = col_names
    if name is not None:
        tmp["groupe"] =  name
    return tmp

In [171]:
unit = [{"unit": '6H',
        "func": 'mean'}, 
        {"unit": '6H',
        "func": 'max'},
       {"unit": '6H',
        "func": 'min'}]

X_n, l_n = create_df_obs(X=X_train,
                         y=y_train,
                         sample=45,
                         add_unit=['day'],
                         source='source',
                         resample={'unit': 'D', 'func': 'min'}, 
                         verbose=False)

In [172]:
X_n

,current,err_down_bip,err_up_bip,olt_recv,rdown,recv,rup,send,temp,volt,day
2000-01-01,15.0,0.0,0.0,NaN,0.0,-28.24,0.0,2.33,34.0,3000.0,D1
2000-01-02,15.0,0.0,0.0,NaN,0.0,-28.24,0.0,2.33,33.0,3000.0,D2
2000-01-03,15.0,0.0,0.0,NaN,0.0,-28.24,0.0,2.33,33.0,3000.0,D3
2000-01-04,15.0,0.0,0.0,NaN,0.0,-28.24,0.0,2.18,33.0,3000.0,D4
2000-01-05,14.0,0.0,0.0,NaN,0.0,-28.24,0.0,2.33,33.0,3000.0,D5
2000-01-06,14.0,0.0,0.0,NaN,0.0,-28.24,0.0,2.33,33.0,3000.0,D6
2000-01-07,14.0,0.0,0.0,NaN,0.0,-28.24,0.0,2.48,33.0,3000.0,D7


In [149]:
test = flatten_df(X_n, name='S')

In [151]:
test

,2000-01-01 00:00:00 current-max,2000-01-01 00:00:00 err_down_bip-max,2000-01-01 00:00:00 err_up_bip-max,2000-01-01 00:00:00 olt_recv-max,2000-01-01 00:00:00 rdown-max,2000-01-01 00:00:00 recv-max,2000-01-01 00:00:00 rup-max,2000-01-01 00:00:00 send-max,2000-01-01 00:00:00 temp-max,2000-01-01 00:00:00 volt-max,...,2000-01-07 00:00:00 err_up_bip-min,2000-01-07 00:00:00 olt_recv-min,2000-01-07 00:00:00 rdown-min,2000-01-07 00:00:00 recv-min,2000-01-07 00:00:00 rup-min,2000-01-07 00:00:00 send-min,2000-01-07 00:00:00 temp-min,2000-01-07 00:00:00 volt-min,day,groupe
0,15,0,0,NaN,4.163,-27.7,0.09,2.76,34,3000,...,0,NaN,0,-28.24,0,2.48,33,3000,D7,S


Exemple du starting kit pour les données d'entrées

In [134]:
pd.DataFrame(X_train.source[:10].reshape(X_train.source[:10].shape[0], -1))

,0,1,2,3,4,5,6,7,8,9,...,6710,6711,6712,6713,6714,6715,6716,6717,6718,6719
0,12.0,4.0,0.0,NaN,0.717,-28.500000,0.075,2.72,37.0,3180.0,...,12.0,6.0,0.0,NaN,0.096,-28.600000,0.063,2.71,37.0,3220.0
1,26.0,293027776.0,0.0,NaN,0.001,-29.209999,0.001,2.76,58.0,3280.0,...,26.0,447741280.0,0.0,NaN,0.000,-29.590000,0.000,2.83,58.0,3280.0
2,11.0,0.0,0.0,NaN,0.187,-27.950001,0.046,1.94,35.0,3280.0,...,11.0,0.0,0.0,NaN,0.114,-27.690001,0.010,1.90,36.0,3280.0
3,8.0,0.0,0.0,NaN,0.001,-28.860001,0.001,2.19,51.0,3280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15.0,0.0,0.0,NaN,0.009,-30.000000,0.012,-30.00,50.0,3000.0,...,14.0,0.0,0.0,NaN,0.861,-30.000000,0.080,-30.00,49.0,3000.0
5,15.0,0.0,7.0,NaN,0.363,-30.959999,0.040,2.45,41.0,3200.0,...,15.0,0.0,0.0,NaN,0.010,-29.580000,0.005,2.45,39.0,3200.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,13.0,0.0,0.0,NaN,3.631,-28.860001,0.077,2.28,44.0,3260.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,12.0,0.0,0.0,-26.58,0.000,-30.969999,0.000,2.08,37.0,3240.0,...,12.0,0.0,0.0,-26.58,0.000,-29.209999,0.000,2.17,37.0,3240.0
9,15.0,1640716.0,391.0,NaN,0.094,-32.220001,0.009,2.76,40.0,3260.0,...,NaN,2698063.0,395.0,NaN,0.001,NaN,0.000,NaN,NaN,NaN


In [130]:
from multiprocessing.pool import ThreadPool

def main(s):
    X, y = create_df_obs(X_train,
                         source='source', 
                         sample=s,
                         extra='',
                         resample={"unit": '6H', "func": 'mean'}, 
                         verbose=False,
                         add_unit=['day'])
    X = flatten_df(X)
    liste_df.append(X)
    return X

liste_df = []
with ThreadPool() as pool:
    start = time.time()
    pool.map(main, range(10000))

print("Time Taken: ", str(time.time()-start))
X = pd.concat(liste_df)

Time Taken:  56.30295205116272


In [135]:
len(X_train.source)

46110

In [139]:
liste_df = []
add_unit = []


for i in range(100):
    start=time.time()
    X, y = create_df_obs(X_train, 
                         source='source', 
                         sample=i,
                         add_unit=[],
                         resample={"unit": 'D', 
                                   "func": 'max'}, 
                         verbose=False)
    X = flatten_df(X, name='S')
    liste_df.append(X)
print("Time Taken: ", str(time.time()-start))
X = pd.concat(liste_df)

Time Taken:  0.0073757171630859375


In [140]:
X

,2000-01-01 00:00:00 current,2000-01-01 00:00:00 err_down_bip,2000-01-01 00:00:00 err_up_bip,2000-01-01 00:00:00 olt_recv,2000-01-01 00:00:00 rdown,2000-01-01 00:00:00 recv,2000-01-01 00:00:00 rup,2000-01-01 00:00:00 send,2000-01-01 00:00:00 temp,2000-01-01 00:00:00 volt,...,2000-01-07 00:00:00 current,2000-01-07 00:00:00 err_down_bip,2000-01-07 00:00:00 err_up_bip,2000-01-07 00:00:00 olt_recv,2000-01-07 00:00:00 rdown,2000-01-07 00:00:00 recv,2000-01-07 00:00:00 rup,2000-01-07 00:00:00 send,2000-01-07 00:00:00 temp,2000-01-07 00:00:00 volt
0,13.0,19.0,0.0,NaN,10.066,-27.700001,1.888,3.01,39.0,3220.0,...,13.0,13.0,0.0,NaN,5.591000,-27.700001,0.758,3.08,38.0,3220.0
0,26.0,299495104.0,0.0,NaN,3.509,-29.209999,0.121,3.34,59.0,3280.0,...,26.0,452120960.0,0.0,NaN,4.863000,-29.590000,0.182,2.90,58.0,3280.0
0,12.0,0.0,0.0,NaN,8.723,-27.690001,0.093,2.19,38.0,3280.0,...,12.0,0.0,0.0,NaN,2.228000,-27.440001,0.091,2.15,39.0,3280.0
0,8.0,0.0,6.0,NaN,0.198,-28.860001,0.008,2.36,51.0,3280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,15.0,0.0,0.0,NaN,7.949,-30.000000,0.347,-30.00,50.0,3000.0,...,14.0,0.0,0.0,NaN,4.843000,-30.000000,0.158,-30.00,50.0,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,14.0,2.0,12.0,NaN,5.982,-29.209999,0.481,2.36,49.0,3320.0,...,13.0,2.0,3.0,NaN,17.271999,-29.209999,0.428,2.36,48.0,3320.0
0,13.0,0.0,0.0,NaN,4.229,-28.240000,0.337,2.43,48.0,3280.0,...,13.0,0.0,0.0,NaN,4.467000,-27.959999,0.233,2.46,47.0,3280.0
0,9.0,1.0,67818.0,NaN,8.991,-28.860001,0.063,2.19,55.0,3260.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,10.0,0.0,22183.0,NaN,0.000,-28.530001,0.000,2.00,33.0,3320.0,...,9.0,0.0,140928.0,NaN,0.001000,-28.530001,0.000,2.00,31.0,3300.0


### Les données

Les données d'entrée sont tridimensionnelles (échantillon, temps, caractéristiques). Le temps a 672 dimensions (4 fois une heure $\times$ 24 heures $\times$ 7 jours). Il contient des valeurs nan, il doit donc être nettoyé.

In [131]:
X_train.source[6].shape

(672, 10)

### Classification

Vous devez soumettre un extracteur de caractéristiques et un classificateur. La fonction transform de l'extracteur de caractéristiques est exécutée sur chaque donnée d'entrée (cible, source, bkg) et les tableaux résultants sont passés aux fonctions fit et predict du classificateur. L'extracteur de caractéristiques du kit de départ remplace nans par zéro, et aplatit la matrice en **(sample, 6720)**.

In [278]:
%%writefile submissions/starting_kit/feature_extractor.py
import numpy as np

class FeatureExtractor:

    def __init__(self):
        pass

    def transform(self, X):
        # Deal with NaNs inplace
        np.nan_to_num(X, copy=False)
        # We flatten the input, originally 3D (sample, time, dim) to
        # 2D (sample, time * dim)
        X = X.reshape(X.shape[0], -1)
        print(X.shape)
        return X


Overwriting submissions/starting_kit/feature_extractor.py


In [43]:
%%writefile submissions/source_rf/classifier.py
from sklearn.ensemble import RandomForestClassifier
from utils.dataset import OpticalDataset, OpticalLabels
from lightgbm import LGBMClassifier

import numpy as np

class Classifier:

    def __init__(self):
        self.clf = LGBMClassifier(
            n_estimators=50, 
            max_depth=20, 
            random_state=44, 
            num_leaves=31,
            n_jobs=-1)
        print(self.clf)

    def fit(self, X_source, X_source_bkg, X_target, X_target_unlabeled,
            X_target_bkg, y_source, y_target):
        self.clf.fit(X_source, y_source)

    def predict_proba(self, X_target, X_target_bkg):
        y_proba = self.clf.predict_proba(X_target)
        return y_proba


Overwriting submissions/source_rf/classifier.py


In [279]:
trained_workflow = problem.workflow.train_submission('submissions/starting_kit', X_train, y_train)
y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)

(46110, 6720)
(50862, 6720)
(438, 6720)
(8202, 6720)
(29592, 6720)
(17758, 6720)
(47275, 6720)


### Scores

In [45]:
ap    = problem.score_types[0]
rec5  = problem.score_types[1]
rec10 = problem.score_types[2]
rec20 = problem.score_types[3]
acc   = problem.score_types[4]
auc   = problem.score_types[5]

In [46]:
print('ap test score    = {}'.format(ap(y_test.target, y_test_pred[:,1])))
print('rec5 test score  = {}'.format(rec5(y_test.target, y_test_pred[:,1])))
print('rec10 test score = {}'.format(rec10(y_test.target, y_test_pred[:,1])))
print('rec20 test score = {}'.format(rec20(y_test.target, y_test_pred[:,1])))
print('acc test score   = {}'.format(acc(y_test.target, y_test_pred.argmax(axis=1))))
print('auc test score   = {}'.format(auc(y_test.target, y_test_pred[:,1])))

ap test score    = 0.1838770011572445
rec5 test score  = 0.08674804121255875
rec10 test score = 0.16564951837062836
rec20 test score = 0.32083696126937866
acc test score   = 0.823234598490821
auc test score   = 0.6293792968994895


### Cross validation

Nous utilisons une validation croisée dix fois (stratifiée lorsque les étiquettes sont disponibles) pour tous les ensembles de données. Dans chaque split, 20% des instances sont dans l'ensemble de validation, à l'exception des données cibles étiquetées qui servent principalement à la validation (pour obtenir une estimation non biaisée des scores de test, évalués entièrement sur des échantillons cibles étiquetés). Nous plaçons vingt points cibles étiquetés dans les splits d'entraînement. La raison en est que lorsque nous étendons nos services à large bande à la ville B, nous pouvons obtenir rapidement un petit ensemble de données étiquetées, mais nous aimerions déployer notre détecteur de défaillance sans attendre deux mois pour recueillir des données comparables à celles de la ville A.

Le schéma de validation croisée (voir `problem.get_cv`) est implémenté dans la classe `TLShuffleSplit` de `external_imports.utils.cv.py`, si vous voulez y regarder de plus près.

Vous êtes libre de jouer avec la coupure train/test et la validation croisée lors du développement de vos modèles mais sachez que nous utiliserons la même configuration sur le serveur officiel que celle du kit RAMP (sur un ensemble différent de quatre campagnes qui ne sera pas disponible pour vous).

La cellule suivante passe par les mêmes étapes que le script d'évaluation officiel (`ramp-test`).

In [69]:
splits = problem.get_cv(X_train, y_train)

In [13]:
splits = problem.get_cv(X_train, y_train)

y_test_preds = []
for fold_i, (train_is, valid_is) in enumerate(splits):
    trained_workflow = problem.workflow.train_submission(
        'submissions/starting_kit', X_train, y_train, train_is)
    X_fold_train = X_train.slice(train_is)
    X_fold_valid = X_train.slice(valid_is)
    
    y_train_pred = problem.workflow.test_submission(trained_workflow, X_fold_train)
    y_valid_pred = problem.workflow.test_submission(trained_workflow, X_fold_valid)
    y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)
    print('-------------------------------------')
    print('training ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(train_is).target, y_train_pred[:,1])))
    print('validation ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(valid_is).target, y_valid_pred[:,1])))
    print('test ap on fold {} = {}'.format(fold_i, ap(y_test.target, y_test_pred[:,1])))
    
    y_test_preds.append(y_test_pred)

-------------------------------------
training ap on fold 0 = 0.30833333333333335
validation ap on fold 0 = 0.2637875964895809
test ap on fold 0 = 0.16218430339780684
-------------------------------------
training ap on fold 1 = 0.21250000000000002
validation ap on fold 1 = 0.2555942077788053
test ap on fold 1 = 0.16361016472786805
-------------------------------------
training ap on fold 2 = 0.2
validation ap on fold 2 = 0.29440601825201235
test ap on fold 2 = 0.1745388926023523
-------------------------------------
training ap on fold 3 = 0.7375
validation ap on fold 3 = 0.28218715512682335
test ap on fold 3 = 0.16904411795376056
-------------------------------------
training ap on fold 4 = 0.21250000000000002
validation ap on fold 4 = 0.24879604051634688
test ap on fold 4 = 0.16172210972525408


KeyboardInterrupt: 

Nous calculons à la fois le score moyen du test et le score de la mise en sac de vos dix modèles. Le classement officiel sera déterminé par le score de test mis en sac (sur des ensembles de données différents de ceux dont vous disposez). Votre score public sera le score de validation mis en sac (le calcul de la moyenne est [légèrement plus compliqué](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/rampwf/utils/combine.py#L56) car nous devons nous occuper correctement des masques de validation croisée). 

In [16]:
bagged_y_pred = np.array(y_test_preds).mean(axis=0)
print('Mean ap score = {}'.format(
    np.mean([ap(y_test.target, y_test_pred[:,1]) for y_test_pred in y_test_preds])))
print('Bagged ap score = {}'.format(
    ap(y_test.target, np.array([y_test_pred for y_test_pred in y_test_preds]).mean(axis=0)[:,1])))

Mean ap score = 0.1662199176814084
Bagged ap score = 0.1688256992369087


## Exemple submissions

Outre le kit de départ, nous vous proposons deux autres exemples de soumissions. L'extracteur de caractéristiques est le même dans les trois. `source_rf` est similaire au kit de départ, mais utilise des arbres plus nombreux et plus profonds, pour obtenir un meilleur score. `target_rf` est une autre soumission extrême qui utilise seulement l'instance d'entraînement de la cible (peu) étiquetée pour apprendre un classificateur. Il a une performance légèrement moins bonne que `source_rf` ce qui signifie que les données sources améliorent le classificateur même si les distributions sources et cibles sont différentes.

### Resultats:
|          | ap             | rec-5         | rec-10         | rec-20         | acc            |  auc           | 
|:---------|:--------------:|:-------------:|:--------------:|:--------------:|:--------------:|:--------------:|   
|source_rf | 0.191 ± 0.0026 | 0.073 ± 0.002 | 0.176 ± 0.0032 | 0.357 ± 0.0075 | 0.84 ± 0.0014  | 0.637 ± 0.0063 | 
|target_rf | 0.163 ± 0.0218 | 0.067 ± 0.0182| 0.138 ± 0.0339 | 0.272 ± 0.0537 | 0.813 ± 0.036  | 0.591 ± 0.0399 | 

La grande question de l'apprentissage par transfert à résoudre est la suivante : **Comment combiner les données cibles à faible biais et à haute variance avec les données sources à faible biais et à haute variance**. D'autres questions auxquelles nous nous attendons à voir des réponses :

1. Peut-on faire un meilleur prétraitement (amputation des données manquantes, utilisation du temps d'une manière plus intelligente) dans l'extracteur de caractéristiques ?
2. Normalement, les données d'arrière-plan (bonnes instances) ne participent pas au scoring, mais elles peuvent informer le classifieur du changement de distribution. Comment utiliser au mieux cette information ?

## Local testing (before submission)

You submission will contain a `feature_extractor.py` implementing a FeatureExtractor class with a `transform` function (no `fit`) and a `classifier.py` implementing a Classifier class with a `fit` and `predict_proba` functions as in the starting kit. You should place it in the `submission/<submission_name>` folder in your RAMP kit folder. To test your submission, go to your RAMP kit folder in the terminal and type
```
ramp-test --submission <submission_name>
```
It will train and test your submission much like we did it above in this notebook, and print the foldwise and summary scores. You can try it also in this notebook:

In [ ]:
!ramp-test --submission target_rf

If you want to have a local leaderboard, use the `--save-output` option when running `ramp-test`, then try `ramp-show leaderboard` with different options. For example:
```
ramp-show leaderboard --mean --metric "['ap','auc']" --step "['valid','test']" --precision 3
```
and
```
ramp-show leaderboard --bagged --metric "['auc']"
```

RAMP also has an experimental hyperopt feature, with random grid search implemented. If you want to use it, type
```
ramp-hyperopt --help
```
and check out the example submission [here](https://github.com/ramp-kits/titanic/tree/hyperopt/submissions/starting_kit_h).